# Oil Environment Code Demo

This problem, adaptved from [here](https://www.pnas.org/content/109/3/764) is a continuous variant of the “Grid World” environment. It comprises of an agent surveying a d-dimensional map in search of hidden “oil deposits”. The world is endowed with an unknown survey function which encodes the probability of observing oil at that specific location. For agents to move to a new location they pay a cost proportional to the distance moved, and surveying the land produces noisy estimates of the true value of that location. In addition, due to varying terrain the true location the agent moves to is perturbed as a function of the state and action.


There is a $d$-dimensional reinforcement learning environment in the space $X = [0, 1]^d$.  The action space $A = [0,1]^d$ corresponding to the ability to attempt to move to any desired location within the state space.  On top of that, there is a corresponding reward function $f_h(x,a)$ for the reward for moving the agent to that location.  Moving also causes an additional cost $\alpha d(x,a)$ scaling with respect to the distance moved.

In this notebook we run a sample experiment for the setting when $d = 1$ and the reward function is taken to be a quadratic.  We compare several heuristics to existing reinforcement learning algorithms.

Here is an example illustrating the problem on a 1 dimensional line:

![Oil_Line_Diagram](diagrams/oil_line_diagram.png)
    
* Assuming a reasonable cost to move, the agent will likely want to move towards the right. If the cost to move is heavily penalized, the agent could chose to stay in place or possibly move to the left.
* Exactly how far the agent moves will be determined by the cost to move
* Finally, the agent may not end up exactly at its target location, as affected by the “terrain”

### Package Installation

In [1]:
import or_suite
import numpy as np

import copy

import os
from stable_baselines3.common.monitor import Monitor
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
import pandas as pd


import gym

### Experimental Parameters


The oil discovery problem has several experiment parameters
* The parameter `epLen`, an int, number of time steps to run the experiment for 
* `nEps` is an int representing the number of episodes. Here it is set to 300. 
* `numIters`, an int, is the number of iterations. Here it is set to 30. 
* `seed` allows random numbers to be generated. 
* `dirPath`, a string, is the location where the data files are stored.
* `deBug`, a bool, prints information to the command line when set true. 
* `save_trajectory`, a bool, saves the trajectory information of the simulation when set to true. 
* `render` renders the algorithm when set to true.
* `pickle` is a bool that saves the information to a pickle file when set to true.

In [17]:
CONFIG =  or_suite.envs.env_configs.oil_environment_default_config

epLen = CONFIG['epLen']
nEps = 300
numIters = 30

epsilon = (nEps * epLen)**(-1 / 4)
action_net = np.arange(start=0, stop=1, step=epsilon)
state_net = np.arange(start=0, stop=1, step=epsilon)

scaling_list = [0.1, 0.3, 1, 5]

DEFAULT_SETTINGS = {'seed': 1, 
                    'recFreq': 1, 
                    'dirPath': '../data/oil/', 
                    'deBug': False, 
                    'nEps': nEps, 
                    'numIters': numIters, 
                    'saveTrajectory': True, 
                    'epLen' : 5,
                    'render': False,
                    'pickle': False
                    }



### Specifying Agent

We specify 6 different agents to compare the effectiveness of each.

* `SB PPO` is Proximal Policy Optimization. When policy is updated, there is a parameter that “clips” each policy update so that action update does not go too far
* `Random` implements the randomized RL algorithm, which selects an action uniformly at random from the action space. In particular, the algorithm stores an internal copy of the environment’s action space and samples uniformly at random from it.
* `AdaQL` is an Adaptive Discretization Model-Free Agent, implemented for enviroments with continuous states and actions using the metric induced by the l_inf norm.
* `AdaMB` is an Adaptive Discretizaiton Model-Based Agent, implemented for enviroments with continuous states and actions using the metric induced by the l_inf norm.
* `Unif QL` is an eNet Model-Based Agent, implemented for enviroments with continuous states and actions using the metric induces by the l_inf norm.
* `Unif MB` is a eNet Model-Free Agent, implemented for enviroments with continuous states and actions using the metric induces by the l_inf norm.

In [18]:
oil_env = gym.make('Oil-v0', config=CONFIG)
mon_env = Monitor(oil_env)
dim = CONFIG['dim']
cost_param = CONFIG['cost_param']
prob = CONFIG['oil_prob']

agents = { 'SB PPO': PPO(MlpPolicy, mon_env, gamma=1, verbose=0, n_steps=epLen),
'Random': or_suite.agents.rl.random.randomAgent(),
'AdaQL': or_suite.agents.rl.ada_ql.AdaptiveDiscretizationQL(epLen, scaling_list[0], True, dim*2),
'AdaMB': or_suite.agents.rl.ada_mb.AdaptiveDiscretizationMB(epLen, scaling_list[0], 0, 2, True, True, dim, dim),
'Unif QL': or_suite.agents.rl.enet_ql.eNetQL(action_net, state_net, epLen, scaling_list[0], (dim,dim)),
'Unif MB': or_suite.agents.rl.enet_mb.eNetMB(action_net, state_net, epLen, scaling_list[0], (dim,dim), 0, False),
}

### Running Algorithm

In [19]:
path_list_line = []
algo_list_line = []
path_list_radar = []
algo_list_radar= []

for agent in agents:
    print(agent)
    DEFAULT_SETTINGS['dirPath'] = '../data/oil_metric_'+str(agent)+'_'+str(dim)+'_'+str(cost_param)+'_'+str(prob.__name__)+'/'
    if agent == 'SB PPO':
        or_suite.utils.run_single_sb_algo(mon_env, agents[agent], DEFAULT_SETTINGS)
    elif agent == 'AdaQL' or agent == 'Unif QL' or agent == 'AdaMB' or agent == 'Unif MB':
        or_suite.utils.run_single_algo_tune(oil_env, agents[agent], scaling_list, DEFAULT_SETTINGS)
    else:
        or_suite.utils.run_single_algo(oil_env, agents[agent], DEFAULT_SETTINGS)

    path_list_line.append('../data/oil_metric_'+str(agent)+'_'+str(dim)+'_'+str(cost_param)+'_'+str(prob.__name__))
    algo_list_line.append(str(agent))
    if agent != 'SB PPO':
        path_list_radar.append('../data/oil_metric_'+str(agent)+'_'+str(dim)+'_'+str(cost_param)+'_'+str(prob.__name__))
        algo_list_radar.append(str(agent))

SB PPO
New Experiment Run
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
[3.9656014429890805, 3.6777668045343375, 4.183091573928946, 3.635085052198403, 3.213399698094551, 3.6378385459579823, 3.6585187036606377, 3.046391411899335, 2.9808309664465664, 3.8326004668142337, 4.367879441171443, 4.262726787837815, 3.177231391004868, 4.073162481090812, 2.330133568391247, 3.688685742332587, 4.133871364759286, 4.487988979950675, 4.861030376166577, 3.536148551160709, 3.898351722376522, 2.5861183903753213, 4.4943902158872, 3.5077156409840016, 4.59108347804297, 3.9110341493988465, 3.6780956574642936, 4.367879441171443, 5.0, 5.0, 4.426294422659

Writing to file data.csv
AdaQL
Chosen parameters: 0.1
Writing to file data.csv
0.1
AdaMB
Chosen parameters: 0.3
Writing to file data.csv
0.3
Unif QL
Chosen parameters: 0.1
Writing to file data.csv
0.1
Unif MB
Chosen parameters: 0.1
Writing to file data.csv
0.1


In [20]:
fig_path = '../figures/'
fig_name = 'oil_metric'+'_'+str(dim)+'_'+str(cost_param)+'_'+str(prob.__name__)+'_line_plot'+'.pdf'
or_suite.plots.plot_line_plots(path_list_line, algo_list_line, fig_path, fig_name, int(nEps / 40)+1)

additional_metric = {}
fig_name = 'oil_metric'+'_'+str(dim)+'_'+str(cost_param)+'_'+str(prob.__name__)+'_radar_plot'+'.pdf'
or_suite.plots.plot_radar_plots(path_list_radar, algo_list_radar,
fig_path, fig_name,
additional_metric
)

  Algorithm    Reward      Time        Space
0    Random  3.691599  6.571526 -3646.000000
1     AdaQL  4.832878  5.703069 -3747.600000
2     AdaMB  4.774606  5.839385 -3809.733333
3   Unif QL  4.723664  6.147670 -3480.800000
4   Unif MB  5.000000  5.150839 -3540.000000


In [1]:
from IPython.display import IFrame
IFrame("../figures/oil_metric_1_0_<lambda>_line_plot.pdf", width=600, height=280)

In [22]:
IFrame("../figures/oil_metric_1_0_<lambda>_radar_plot.pdf", width=600, height=450)

Here we see the uniform discretization model based algorithm performs the best with a minimal time complexity for evaluating the algorithm.